## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-08-10-50-57 +0000,nyt,Inside China’s Quest to Defy Aging with Longev...,https://www.nytimes.com/2025/11/08/world/asia/...
1,2025-11-08-10-39-08 +0000,wapo,The Farmers’ Almanac reaches the end of the ro...,https://www.washingtonpost.com/nation/2025/11/...
2,2025-11-08-10-30-00 +0000,wsj,Alaska’s New Mining Rush Chases Something More...,https://www.wsj.com/us-news/alaskas-new-mining...
3,2025-11-08-10-30-00 +0000,wsj,Meet the Advisers Who Will Shape Mamdani’s Adm...,https://www.wsj.com/us-news/zohran-mamdani-adv...
4,2025-11-08-10-30-00 +0000,wsj,Disney-YouTube TV Blackout Has Customers Scram...,https://www.wsj.com/business/media/disney-yout...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2326/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
107,trump,47
142,up,14
89,shutdown,14
421,court,11
84,flight,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
76,2025-11-08-02-02-29 +0000,nypost,Supreme Court allows Trump admin to temporaril...,https://nypost.com/2025/11/07/us-news/trump-ad...,113
294,2025-11-07-12-40-35 +0000,cbc,Trump administration fights court ruling to pa...,https://www.cbc.ca/news/world/us-ri-court-ruli...,105
78,2025-11-08-02-00-00 +0000,wsj,Republicans in Washington got a clear takeaway...,https://www.wsj.com/politics/policy/trump-pric...,104
66,2025-11-08-03-45-03 +0000,nyt,Trump Administration Appeals to Supreme Court ...,https://www.nytimes.com/2025/11/07/us/politics...,101
68,2025-11-08-02-59-00 +0000,wsj,Supreme Court Pauses Order for Trump Administr...,https://www.wsj.com/us-news/law/supreme-court-...,100


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
76,113,2025-11-08-02-02-29 +0000,nypost,Supreme Court allows Trump admin to temporaril...,https://nypost.com/2025/11/07/us-news/trump-ad...
314,65,2025-11-07-11-00-00 +0000,latimes,Your flight was canceled amid FAA's cuts to ai...,https://www.latimes.com/california/story/2025-...
78,57,2025-11-08-02-00-00 +0000,wsj,Republicans in Washington got a clear takeaway...,https://www.wsj.com/politics/policy/trump-pric...
137,53,2025-11-07-22-06-00 +0000,wsj,Defense Secretary Pete Hegseth said that the P...,https://www.wsj.com/politics/national-security...
275,40,2025-11-07-14-11-00 +0000,wsj,Senate Majority Leader John Thune expects Sena...,https://www.wsj.com/politics/policy/government...
101,39,2025-11-07-23-56-29 +0000,nypost,Tesla rival Rivian gives CEO RJ Scaringe a Mus...,https://nypost.com/2025/11/07/business/rivian-...
247,38,2025-11-07-16-04-00 +0000,wsj,Heritage Foundation president Kevin Roberts de...,https://www.wsj.com/politics/policy/the-crack-...
98,35,2025-11-08-00-06-04 +0000,nypost,Michigan woman captured hurling piping hot cof...,https://nypost.com/2025/11/07/us-news/michigan...
85,34,2025-11-08-01-13-34 +0000,nypost,Mayor-elect Zohran Mamdani schmoozes with NY D...,https://nypost.com/2025/11/07/us-news/zohran-m...
35,33,2025-11-08-08-51-00 +0000,wsj,UPS and FedEx Ground MD-11 Fleets After Deadly...,https://www.wsj.com/us-news/ups-and-fedex-grou...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
